# Machine Learning - IC

## Importando os Dados:

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import mlab
from scipy import signal
from scipy.fft import fftshift
from tqdm import tqdm
from scipy.signal import butter, lfilter
import seaborn as sns
import os
import math as m
import shutil
import random
import statistics as stc
from statistics import variance
import scipy.stats as stats
from scipy.stats import kurtosis
import plotly.express as px

In [ ]:
#Imports Sklearn

from sklearn import tree

#Preprocessamento

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


#Regressão

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

#Classificação

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#Avaliação e otimização

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#Clustering

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

#Dimensionality Reduction

from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/DadosML/'

In [ ]:
df_ML = pd.read_csv(path+ 'DataframeML.csv',index_col=0)

In [ ]:
#Mostrando o Heading
df_ML.head()

In [ ]:
# Imprimir Tabela
df_ML

In [ ]:
#Conferindo Type de cada coluna:
df_ML.info()

In [ ]:
#Primeiras Informações:
df_ML.describe()

In [ ]:
#Conferindo NANs:
df_ML.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_ML.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln=list(df_ML.columns)
#print(coln, "\n")

#Separando Variaveis Temporais:

c_T = coln[1:74]

#Separando os nomes das Colunas com FFT's: 'X.iloc[:,(1250*2)+73:(73+(1250*(2+1)))]'

c_F_a1_x=coln[74:1324]
c_F_a1_y=coln[1324:2574]
c_F_a1_z=coln[2574:3824]

c_F_a2_x=coln[3824:5074]
c_F_a2_y=coln[5074:6324]
c_F_a2_z=coln[6324:7574]

c_F_a3_x=coln[7574:8824]
c_F_a3_y=coln[8824:10074]
c_F_a3_z=coln[10074:11324]

c_F_g1_x=coln[11324:12574]
c_F_g1_y=coln[12574:13824]
c_F_g1_z=coln[13824:15074]

c_F_g2_x=coln[15074:16324]
c_F_g2_y=coln[16324:17574]
c_F_g2_z=coln[17574:18824]

c_F_g3_x=coln[18824:20074]
c_F_g3_y=coln[20074:21324]
c_F_g3_z=coln[21324:22574]



print(c_T)
print(c_F_a1_x)
print(c_F_a1_y)
print(c_F_a1_z)
print(c_F_a2_x)
print(c_F_a2_y)
print(c_F_a2_z)
print(c_F_a3_x)
print(c_F_a3_y)
print(c_F_a3_z)
print(c_F_g1_x)
print(c_F_g1_y)
print(c_F_g1_z)
print(c_F_g2_x)
print(c_F_g2_y)
print(c_F_g2_z)
print(c_F_g3_x)
print(c_F_g3_y)
print(c_F_g3_z)

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
Y

In [ ]:
falhas=['Normal','Desbalanceamento Estático', 'Desbalanceamento Dinâmico']

In [ ]:
#Dividindo Dados No Tempo:

X_tempo = X[c_T]

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
#Teste para ver se criou o Sub-dataset
X_fft_g1_z

# Analisando Dados Sem PCA

* Nosso grande problema é a quantidade de dados a serem analisados para prever o tipo de defeito no equipamento!
* Portanto na EDA, vamos realizar uma Feature Selection partindo do principio de Backward Elimination Method e Forward Selection Method.
* OBS: Para escolher as melhores features, pode-se usar vários modelos para comparar (KNN, RF, ...). Vamos usar Random Forest.

## Dados só Temporais

### RapidMiner: Tabela Completa de Tempo

In [ ]:
#Dataframe Completo com Tempo:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_X_tempo=pd.DataFrame(X_tempo, columns = c_T)

In [ ]:
df_Y

In [ ]:
df_X_tempo

In [ ]:
frames=[df_Y,df_X_tempo]
df_Completo_tempo=pd.concat(frames,axis=1)

In [ ]:
df_Completo_tempo

In [ ]:
df_Completo_tempo.to_csv('DataframeML_Completo_Tempo.csv')

### RF

In [ ]:
X_tempo

In [ ]:
Y

In [ ]:
#Vamos fazer uma Random Forest para Toda o Dataframe, sem PCA e Scaler:
Y = LabelEncoder().fit_transform(Y)

sc = StandardScaler()
X_tsc=sc.fit_transform(X_tempo)


X_train, X_test, y_train, y_test = train_test_split(X_tsc, Y, test_size = 0.3, random_state = 0)

RFC= RandomForestClassifier(max_depth= 100, n_estimators = 200, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)

In [ ]:
train_score_rf=RFC.score(X_train,y_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC.score(X_test,y_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
RFC.feature_importances_

In [ ]:
np.max(RFC.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_tempo.columns,
    "Imp": RFC.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_train,RFC.predict(X_train))
confmatrix_test=confusion_matrix(y_test,RFC.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### SVC

In [ ]:
Y = LabelEncoder().fit_transform(Y)

Y

In [ ]:
X_tempo

In [ ]:
sc = StandardScaler()
X_tsc=sc.fit_transform(X_tempo)


X_train, X_test, y_train, y_test = train_test_split(X_tsc, Y, test_size = 0.3, random_state = 0)

In [ ]:
SVC = SVC(kernel = 'linear', random_state = 0)
SVC.fit(X_train, y_train)

In [ ]:
y_pred = SVC.predict(X_test)

In [ ]:
train_score_svc=SVC.score(X_train,y_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC.score(X_test,y_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_train,SVC.predict(X_train))
confmatrix_test=confusion_matrix(y_test,SVC.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

## Dados Com TEMPO (Completo)

### RF

In [ ]:
#Vamos fazer uma Random Forest para Toda o Dataframe, sem PCA e Scaler:
Y = LabelEncoder().fit_transform(Y)

sc = StandardScaler()
X_sc=sc.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_sc, Y, test_size = 0.3, random_state = 0)

RFC= RandomForestClassifier(max_depth= 1000, n_estimators = 2000, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)

In [ ]:
train_score_rf=RFC.score(X_train,y_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC.score(X_test,y_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
RFC.feature_importances_

In [ ]:
np.max(RFC.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X.columns,
    "Imp": RFC.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_train,RFC.predict(X_train))
confmatrix_test=confusion_matrix(y_test,RFC.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### SVC

In [ ]:
Y = LabelEncoder().fit_transform(Y)

Y

In [ ]:
sc = StandardScaler()
X_sc=sc.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_sc, Y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.svm import SVC
SVC = SVC(kernel = 'linear', random_state = 0)
SVC.fit(X_train, y_train)

In [ ]:
y_pred = SVC.predict(X_test)

In [ ]:
train_score_svc=SVC.score(X_train,y_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC.score(X_test,y_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_train,SVC.predict(X_train))
confmatrix_test=confusion_matrix(y_test,SVC.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Tentando Usando Backward Feature Selection

In [ ]:
#https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/
#https://www.youtube.com/watch?v=nh2IGC_kLWw
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html
#https://www.yourdatateacher.com/2023/02/15/a-practical-introduction-to-sequential-feature-selection/

#from sklearn.feature_selection import SequentialFeatureSelector as SFS

#backward_feature_selection = SFS(RFC,n_features_to_select=100,direction='backward',scoring='accuracy',cv=5).fit_transform(X_train,y_train)
#forward_feature_selection = SFS(RFC,n_features_to_select=8,direction='forward',scoring='accuracy',cv=5).fit_transform(X_train,y_train)

## Dados sem Tempo

### Tabela Geral sem Tempo

In [ ]:
df_tr=df_ML

In [ ]:
df_tr

In [ ]:
df_tr.drop(c_T[1:],axis=1,inplace=True)

In [ ]:
df_tr

In [ ]:
df_tr.to_csv('DataframeML_Completo_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_tr=list(df_tr)

#Definindo as Features (X's):
features=coln_tr[1:]
X=df_tr[features]

#Definindo os Targets (Y's):

targets=coln_tr[0] #Definindo lista de targets
Y=df_tr[targets]

### RF

In [ ]:
X

In [ ]:
Y

In [ ]:
#Vamos fazer uma Random Forest para Toda o Dataframe, sem PCA e Scaler:
Y = LabelEncoder().fit_transform(Y)

sc = StandardScaler()
X_sc=sc.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_sc, Y, test_size = 0.3, random_state = 0)

RFC= RandomForestClassifier(max_depth= 1000, n_estimators = 2000, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)

In [ ]:
train_score_rf=RFC.score(X_train,y_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC.score(X_test,y_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
RFC.feature_importances_

In [ ]:
np.max(RFC.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X.columns,
    "Imp": RFC.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_train,RFC.predict(X_train))
confmatrix_test=confusion_matrix(y_test,RFC.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### SVC

In [ ]:
Y = LabelEncoder().fit_transform(Y)

Y

In [ ]:
X

In [ ]:
sc = StandardScaler()
X_sc=sc.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_sc, Y, test_size = 0.3, random_state = 0)

In [ ]:
from sklearn.svm import SVC

SVC1 = SVC(kernel = 'linear', random_state = 0)
SVC1.fit(X_train, y_train)

In [ ]:
y_pred = SVC1.predict(X_test)

In [ ]:
train_score_svc=SVC1.score(X_train,y_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC1.score(X_test,y_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_test, y_pred)
print('accuracy_score %:',accuracy_score(y_test, y_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_train,SVC1.predict(X_train))
confmatrix_test=confusion_matrix(y_test,SVC1.predict(X_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

# PCA

Como temos em torno de 22.500 colunas de features, temos que reduzir essa dimensionalidade !

Observe que:

* Muitos dados, principalmente das FFT's não tem significância, ou seja, são inapropriados para a análise.
* Também, deve-se tomar cuidado com o Underfitting, pois tem-se uma excessividade enorme de features

Assim, recorre-se ao PCA (Principal Component Analysis) para reduzir, no caso, as colunas de FFT. Divide-se em algumas arquiteturas:

1. X Colunas, com 100% de variância, Para Cada FFT
2. Y Colunas, com 90% de variância, Para Cada FFT
3. Z Colunas, com 80% de variância, Para Cada FFT
4. W Colunas, com 70% de variância, Para Cada FFT

Lembrando que cada FFT tem 1250 pontos, temos 3 mancais / senores, cada um medindo 6 features ---> 3 x 6 = 18 Features de cada sensore ---> 18 x 1250 = 22500 pontos totais.

* Temos que aplicar a escala nos dados para aplicar o PCA ! Não ficar enviesado

In [ ]:
df_ML = pd.read_csv(path+ 'DataframeML.csv',index_col=0)

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft:
X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
#Testando:
X_fft_g1_z

In [ ]:
# Aplicando PCA com porcentagem de Variância: Quão significativo são os dados (Total Explaned Variance)

pca = PCA(0.5)  #---> SÓ MUDAR AQUI, QUE MUDA PARA TODO O RESTO !

#Aplicando o PCA para cada FFT:

#PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
#PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
#PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

#PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
#PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
#PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

#PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
#PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
#PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

#PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
#PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
#PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

#PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
#PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
#PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

#PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
#PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
#PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)


#https://stackabuse.com/implementing-pca-in-python-with-scikit-learn/
#https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
#https://builtin.com/machine-learning/pca-in-python
#https://www.jcchouinard.com/pca-with-python/
#https://www.datacamp.com/tutorial/principal-component-analysis-in-python
#https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html
#Último fala sobre filtro de ruido -> PCA faz isso !

In [ ]:
PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for X_fft_a1_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a1_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a1_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a1_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a1_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a1_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a1_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a1_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a1_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a1_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a1_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a1_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a2_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a2_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a2_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a2_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a2_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a2_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a2_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a2_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a2_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a2_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a2_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a2_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a3_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a3_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a3_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a3_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a3_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a3_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a3_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a3_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_a3_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_a3_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_a3_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_a3_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g1_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g1_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g1_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g1_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g1_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g1_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g1_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g1_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g1_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g1_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g1_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g1_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g2_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g2_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g2_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g2_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g2_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g2_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g2_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g2_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g2_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g2_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g2_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g2_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g3_x"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g3_x[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g3_x');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g3_x,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g3_y"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g3_y[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g3_y');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g3_y,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X_fft_g3_z"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X_fft_g3_z[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X_fft_g3_z');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_F_g3_z,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

In [ ]:
# Vendo quantas Colunas são necessarias deixar para obter X% das informações, via PCA:

print('PCA_X_fft_a1_x')
#print(PCA_X_fft_a1_x)
print(len(PCA_X_fft_a1_x[0]))

print()
print()

print('PCA_X_fft_a1_y')
#print(PCA_X_fft_a1_y)
print(len(PCA_X_fft_a1_y[0]))

print()
print()

print('PCA_X_fft_a1_z')
#print(PCA_X_fft_a1_z)
print(len(PCA_X_fft_a1_z[0]))

print()
print()

print('PCA_X_fft_a2_x')
#print(PCA_X_fft_a2_x)
print(len(PCA_X_fft_a2_x[0]))

print()
print()

print('PCA_X_fft_a2_y')
#print(PCA_X_fft_a2_y)
print(len(PCA_X_fft_a2_y[0]))

print()
print()

print('PCA_X_fft_a2_z')
#print(PCA_X_fft_a2_z)
print(len(PCA_X_fft_a2_z[0]))

print()
print()

print('PCA_X_fft_a3_x')
#print(PCA_X_fft_a3_x)
print(len(PCA_X_fft_a3_x[0]))

print()
print()

print('PCA_X_fft_a3_y')
#print(PCA_X_fft_a3_y)
print(len(PCA_X_fft_a3_y[0]))

print()
print()

print('PCA_X_fft_a3_z')
#print(PCA_X_fft_a3_z)
print(len(PCA_X_fft_a3_z[0]))

print()
print()

print('PCA_X_fft_g1_x')
#print(PCA_X_fft_g1_x)
print(len(PCA_X_fft_g1_x[0]))

print()
print()

print('PCA_X_fft_g1_y')
#print(PCA_X_fft_g1_y)
print(len(PCA_X_fft_g1_y[0]))

print()
print()

print('PCA_X_fft_g1_z')
#print(PCA_X_fft_g1_z)
print(len(PCA_X_fft_g1_z[0]))

print()
print()

print('PCA_X_fft_g2_x')
#print(PCA_X_fft_g2_x)
print(len(PCA_X_fft_g2_x[0]))

print()
print()

print('PCA_X_fft_g2_y')
#print(PCA_X_fft_g2_y)
print(len(PCA_X_fft_g2_y[0]))

print()
print()

print('PCA_X_fft_g2_z')
#print(PCA_X_fft_g2_z)
print(len(PCA_X_fft_g2_z[0]))

print()
print()

print('PCA_X_fft_g3_x')
#print(PCA_X_fft_g3_x)
print(len(PCA_X_fft_g3_x[0]))

print()
print()

print('PCA_X_fft_g3_y')
#print(PCA_X_fft_g3_y)
print(len(PCA_X_fft_g3_y[0]))

print()
print()

print('PCA_X_fft_g3_z')
#print(PCA_X_fft_g3_z)
print(len(PCA_X_fft_g3_z[0]))

Pegando o maior número de colunas para cada porcentagem do PCA. Aqui simplismente, comparamos os resultados de cada sensor, mancal e eixo e pegamos o PCA com moir número de colunas:

* PCA (1) -> 380 Colunas
* PCA (0.999) -> 364 Colunas
* PCA (0.99) -> 298 Colunas
* PCA (0.98) -> 263 Colunas
* PCA (0.95) -> 208 Colunas
* PCA (0.90) -> 160 Colunas
* PCA (0.80) -> 106 Colunas
* PCA (0.70) -> 71 Colunas
* PCA (0.60) -> 47 Colunas
* PCA (0.50) -> 29 Colunas

------------------------
Agora testando com TODO o X, sem fazer distinção de nada, loucuras

In [ ]:
X

In [ ]:
c_X=X.columns

In [ ]:
X = sc.fit_transform(X)

In [ ]:
#Aplicando o PCA para cada FFT:
PCA_X = pca.fit_transform(X)

In [ ]:
print('PCA_X')
#print(PCA_X)
print(len(PCA_X[0]))

In [ ]:
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "Number of Components", "y": "Explained Variance"},
    title="Explained Variance for PCA_X"
)

In [ ]:
plt.figure(figsize=(15,8))
df_var_explained = pd.DataFrame({'var':pca.explained_variance_ratio_,'PC':[str(i) for i in range (1,len(PCA_X[0])+1)]})
sns.barplot(x='PC',y="var", data=df_var_explained, color="c").set(title='Variance and PCA for PCA_X');

In [ ]:
#https://towardsdatascience.com/pca-clearly-explained-how-when-why-to-use-it-and-feature-importance-a-guide-in-python-7c274582c37e
#PCA.COMPONENTS mostra a importancia de cada variavel (coluna) para realizar cada PCA. Ou seja, cada linha é um PCA e cada coluna é a importância da variavel "FFT_5_acc_X"


for i in range(0,len(pca.components_)):
  print(f'PCA{i+1} \n')
  print(f'PCA{i+1} : {abs( pca.components_ )[i,:]}')
  pca_df = pd.DataFrame({"Variavel": c_X,"Importância da Variavel para o PCA": abs( pca.components_ )[i,:]})
  print(pca_df.sort_values(by="Importância da Variavel para o PCA", ascending=False)[0:10])

# Arquitetura 1 :
PCA (1)-> 380 Colunas

In [ ]:
df_ML = pd.read_csv(path+ 'DataframeML.csv',index_col=0)
from sklearn.svm import SVC

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA() # 100% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc1=pd.concat(frames,axis=1)

In [ ]:
df_arc1

In [ ]:
df_arc1.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc1.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc1.to_csv('DataframeML_ARC1_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc1=list(df_arc1)
print(coln_arc1, "\n")

print(len(coln_arc1))
#Definindo as Features (X's):
features=coln_arc1[1:]
X_arc1=df_arc1[features]

print(coln_arc1[1:])
#Definindo os Targets (Y's):

targets=coln_arc1[0] #Definindo lista de targets
Y_arc1=df_arc1[targets]

In [ ]:
X_arc1

In [ ]:
Y_arc1

## Random Forest (COM TEMPO)

In [ ]:
X_arc1

In [ ]:
Y_arc1 = LabelEncoder().fit_transform(Y_arc1)

Y_arc1

In [ ]:
X_arc1_train, X_arc1_test, y_arc1_train, y_arc1_test = train_test_split(X_arc1, Y_arc1, test_size = 0.3, random_state = 0)

RFC_arc1 = RandomForestClassifier(max_depth= 500, n_estimators = 500, random_state = 0)
RFC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = RFC_arc1.predict(X_arc1_test)

In [ ]:
train_score_rf=RFC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
RFC_arc1.feature_importances_

In [ ]:
np.max(RFC_arc1.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc1_train.columns,
    "Imp": RFC_arc1.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,RFC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,RFC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc1 = {
'n_estimators': [100,250,500,1000,1500,2000],
'max_depth' : [50,125,250,500,750],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 1:")

grid_arc1 = GridSearchCV(RFC_arc1 ,params_arc1, scoring='accuracy',cv=5)
grid_arc1.fit(X_arc1_train,y_arc1_train)

print(grid_arc1.best_params_, grid_arc1.best_score_)
rf_best_arc1 = grid_arc1.best_estimator_

y_arc1_pred = rf_best_arc1.predict(X_arc1_test)

accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

print(rf_best_arc1.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc1_train.columns,
    "Imp": rf_best_arc1.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc1

In [ ]:
Y_arc1 = LabelEncoder().fit_transform(Y_arc1)

Y_arc1

In [ ]:
X_arc1_train, X_arc1_test, y_arc1_train, y_arc1_test = train_test_split(X_arc1, Y_arc1, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc1 = SVC(kernel = 'linear', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc1 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc1 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc1 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc1_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc1_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc1_SVC_Linear,params_arc1_SVC_RS]:
  print("Otimização Dados:")
  grid_arc1_SVC = GridSearchCV(SVC_arc1 ,params, scoring='accuracy')
  grid_arc1_SVC.fit(X_arc1_train,y_arc1_train)
  print(grid_arc1_SVC.best_params_, grid_arc1_SVC.best_score_)
  rf_best_arc1_SVC = grid_arc1_SVC.best_estimator_

  y_arc1_pred_SVC = rf_best_arc1_SVC.predict(X_arc1_test)

  accuracy_score(y_arc1_test, y_arc1_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc1

In [ ]:
df_arc1.drop(c_X_tempo[1:],axis=1, inplace=True)

In [ ]:
df_arc1

In [ ]:
df_arc1.to_csv('DataframeML_ARC1_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc1=list(df_arc1)
print(coln_arc1, "\n")

print(len(coln_arc1))
#Definindo as Features (X's):
features=coln_arc1[1:]
X_arc1=df_arc1[features]

print(coln_arc1[1:])
#Definindo os Targets (Y's):

targets=coln_arc1[0] #Definindo lista de targets
Y_arc1=df_arc1[targets]

In [ ]:
Y_arc1

In [ ]:
X_arc1

## Random Forest (SEM TEMPO)

In [ ]:
X_arc1

In [ ]:
Y_arc1 = LabelEncoder().fit_transform(Y_arc1)

Y_arc1

In [ ]:
X_arc1_train, X_arc1_test, y_arc1_train, y_arc1_test = train_test_split(X_arc1, Y_arc1, test_size = 0.3, random_state = 0)

In [ ]:
RFC_arc1 = RandomForestClassifier(max_depth= 250, n_estimators = 1000, random_state = 0)
RFC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = RFC_arc1.predict(X_arc1_test)

In [ ]:
train_score_rf=RFC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
RFC_arc1.feature_importances_

In [ ]:
np.max(RFC_arc1.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc1_train.columns,
    "Imp": RFC_arc1.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,RFC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,RFC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc1 = {
'n_estimators': [100,250,500,1000,1500,2000],
'max_depth' : [50,125,250,500,750],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 1 Sem Tempo:")

grid_arc1 = GridSearchCV(RFC_arc1 ,params_arc1, scoring='accuracy',cv=5) #43 min
grid_arc1.fit(X_arc1_train,y_arc1_train)

print(grid_arc1.best_params_, grid_arc1.best_score_)
rf_best_arc1 = grid_arc1.best_estimator_

y_arc1_pred = rf_best_arc1.predict(X_arc1_test)

accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

print(rf_best_arc1.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc1_train.columns,
    "Imp": rf_best_arc1.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc1

In [ ]:
Y_arc1 = LabelEncoder().fit_transform(Y_arc1)

Y_arc1

In [ ]:
X_arc1_train, X_arc1_test, y_arc1_train, y_arc1_test = train_test_split(X_arc1, Y_arc1, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc1 = SVC(kernel = 'linear', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc1 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc1 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc1 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc1.fit(X_arc1_train, y_arc1_train)

In [ ]:
y_arc1_pred = SVC_arc1.predict(X_arc1_test)

In [ ]:
train_score_svc=SVC_arc1.score(X_arc1_train,y_arc1_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc1.score(X_arc1_test,y_arc1_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc1_test, y_arc1_pred)
print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc1_train,SVC_arc1.predict(X_arc1_train))
confmatrix_test=confusion_matrix(y_arc1_test,SVC_arc1.predict(X_arc1_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc1_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc1_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc1_SVC_Linear,params_arc1_SVC_RS]:
  print("Otimização Dados:")
  grid_arc1_SVC = GridSearchCV(SVC_arc1 ,params, scoring='accuracy')
  grid_arc1_SVC.fit(X_arc1_train,y_arc1_train)
  print(grid_arc1_SVC.best_params_, grid_arc1_SVC.best_score_)
  rf_best_arc1_SVC = grid_arc1_SVC.best_estimator_

  y_arc1_pred_SVC = rf_best_arc1_SVC.predict(X_arc1_test)

  accuracy_score(y_arc1_test, y_arc1_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc1_test, y_arc1_pred_SVC)*100)
  print()

# Arquitetura 2 :
PCA (0.90)-> 160 Colunas

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.90) # 90% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc2=pd.concat(frames,axis=1)

In [ ]:
df_arc2

In [ ]:
df_arc2.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc2.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc2.to_csv('DataframeML_ARC2_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc2=list(df_arc2)
print(coln_arc2, "\n")

print(len(coln_arc2))
#Definindo as Features (X's):
features=coln_arc2[1:]
X_arc2=df_arc2[features]

print(coln_arc2[1:])
#Definindo os Targets (Y's):

targets=coln_arc2[0] #Definindo lista de targets
Y_arc2=df_arc2[targets]

In [ ]:
X_arc2

In [ ]:
Y_arc2

## Random Forest (COM TEMPO)

In [ ]:
X_arc2

In [ ]:
Y_arc2 = LabelEncoder().fit_transform(Y_arc2)

Y_arc2

In [ ]:
X_arc2_train, X_arc2_test, y_arc2_train, y_arc2_test = train_test_split(X_arc2, Y_arc2, test_size = 0.3, random_state = 0)

RFC_arc2 = RandomForestClassifier(max_depth= 100, n_estimators = 100, random_state = 0)
RFC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = RFC_arc2.predict(X_arc2_test)

In [ ]:
train_score_rf=RFC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
RFC_arc2.feature_importances_

In [ ]:
np.max(RFC_arc2.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc2_train.columns,
    "Imp": RFC_arc2.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,RFC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,RFC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc2 = {
'n_estimators': [50,100,250,500,1000],
'max_depth' : [25,50,125,250,500],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 2:")

grid_arc2 = GridSearchCV(RFC_arc2 ,params_arc2, scoring='accuracy',cv=5) #43 min
grid_arc2.fit(X_arc2_train,y_arc2_train)

print(grid_arc2.best_params_, grid_arc2.best_score_)
rf_best_arc2 = grid_arc2.best_estimator_

y_arc2_pred = rf_best_arc2.predict(X_arc2_test)

accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

print(rf_best_arc2.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc2_train.columns,
    "Imp": rf_best_arc2.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc2

In [ ]:
Y_arc2 = LabelEncoder().fit_transform(Y_arc2)

Y_arc2

In [ ]:
X_arc2_train, X_arc2_test, y_arc2_train, y_arc2_test = train_test_split(X_arc2, Y_arc2, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc2 = SVC(kernel = 'linear', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc2 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc2 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc2 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc2_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc2_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc2_SVC_Linear,params_arc2_SVC_RS]:
  print("Otimização Dados Arquitetura 2:")
  grid_arc2_SVC = GridSearchCV(SVC_arc2 ,params, scoring='accuracy')
  grid_arc2_SVC.fit(X_arc2_train,y_arc2_train)
  print(grid_arc2_SVC.best_params_, grid_arc2_SVC.best_score_)
  rf_best_arc2_SVC = grid_arc2_SVC.best_estimator_

  y_arc2_pred_SVC = rf_best_arc2_SVC.predict(X_arc2_test)

  accuracy_score(y_arc2_test, y_arc2_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc2

In [ ]:
df_arc2.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc2

In [ ]:
df_arc2.to_csv('DataframeML_ARC2_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc2=list(df_arc2)
print(coln_arc2, "\n")

print(len(coln_arc2))
#Definindo as Features (X's):
features=coln_arc2[1:]
X_arc2=df_arc2[features]

print(coln_arc2[1:])
#Definindo os Targets (Y's):

targets=coln_arc2[0] #Definindo lista de targets
Y_arc2=df_arc2[targets]

In [ ]:
Y_arc2

In [ ]:
X_arc2

## Random Forest (SEM TEMPO)

In [ ]:
X_arc2

In [ ]:
Y_arc2 = LabelEncoder().fit_transform(Y_arc2)

Y_arc2

In [ ]:
X_arc2_train, X_arc2_test, y_arc2_train, y_arc2_test = train_test_split(X_arc2, Y_arc2, test_size = 0.3, random_state = 0)

In [ ]:
RFC_arc2 = RandomForestClassifier(max_depth= 500, n_estimators = 1000, random_state = 0)
RFC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = RFC_arc2.predict(X_arc2_test)

In [ ]:
train_score_rf=RFC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
RFC_arc2.feature_importances_

In [ ]:
np.max(RFC_arc2.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc2_train.columns,
    "Imp": RFC_arc2.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,RFC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,RFC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc2 = {
'n_estimators': [50,100,250,500,1000],
'max_depth' : [25,50,125,250,500],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 2 Sem Tempo:")

grid_arc2 = GridSearchCV(RFC_arc2 ,params_arc2, scoring='accuracy',cv=5)
grid_arc2.fit(X_arc2_train,y_arc2_train)

print(grid_arc2.best_params_, grid_arc2.best_score_)
rf_best_arc2 = grid_arc2.best_estimator_

y_arc2_pred = rf_best_arc2.predict(X_arc2_test)

accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

print(rf_best_arc2.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc2_train.columns,
    "Imp": rf_best_arc2.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc2

In [ ]:
Y_arc2 = LabelEncoder().fit_transform(Y_arc2)

Y_arc2

In [ ]:
X_arc2_train, X_arc2_test, y_arc2_train, y_arc2_test = train_test_split(X_arc2, Y_arc2, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc2 = SVC(kernel = 'linear', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc2 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc2 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc2 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc2.fit(X_arc2_train, y_arc2_train)

In [ ]:
y_arc2_pred = SVC_arc2.predict(X_arc2_test)

In [ ]:
train_score_svc=SVC_arc2.score(X_arc2_train,y_arc2_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc2.score(X_arc2_test,y_arc2_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc2_test, y_arc2_pred)
print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc2_train,SVC_arc2.predict(X_arc2_train))
confmatrix_test=confusion_matrix(y_arc2_test,SVC_arc2.predict(X_arc2_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc2_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc2_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc2_SVC_Linear,params_arc2_SVC_RS]:
  print("Otimização Dados Arqutetura 2 - Sem Tempo:")
  grid_arc2_SVC = GridSearchCV(SVC_arc2 ,params, scoring='accuracy')
  grid_arc2_SVC.fit(X_arc2_train,y_arc2_train)
  print(grid_arc2_SVC.best_params_, grid_arc2_SVC.best_score_)
  rf_best_arc2_SVC = grid_arc2_SVC.best_estimator_

  y_arc2_pred_SVC = rf_best_arc2_SVC.predict(X_arc2_test)

  accuracy_score(y_arc2_test, y_arc2_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc2_test, y_arc2_pred_SVC)*100)
  print()

# Arquitetura 3 :
PCA (0.80)-> 106 Colunas

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.80) # 80% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc3=pd.concat(frames,axis=1)

In [ ]:
df_arc3

In [ ]:
df_arc3.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc3.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc3.to_csv('DataframeML_ARC3_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc3=list(df_arc3)
print(coln_arc3, "\n")

print(len(coln_arc3))
#Definindo as Features (X's):
features=coln_arc3[1:]
X_arc3=df_arc3[features]

print(coln_arc3[1:])
#Definindo os Targets (Y's):

targets=coln_arc3[0] #Definindo lista de targets
Y_arc3=df_arc3[targets]

In [ ]:
X_arc3

In [ ]:
Y_arc3

## Random Forest (COM TEMPO)

In [ ]:
X_arc3

In [ ]:
Y_arc3 = LabelEncoder().fit_transform(Y_arc3)

Y_arc3

In [ ]:
X_arc3_train, X_arc3_test, y_arc3_train, y_arc3_test = train_test_split(X_arc3, Y_arc3, test_size = 0.3, random_state = 0)

RFC_arc3 = RandomForestClassifier(max_depth= 20, n_estimators = 20, random_state = 0)
RFC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = RFC_arc3.predict(X_arc3_test)

In [ ]:
train_score_rf=RFC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
RFC_arc3.feature_importances_

In [ ]:
np.max(RFC_arc3.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc3_train.columns,
    "Imp": RFC_arc3.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,RFC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,RFC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc3 = {
'n_estimators': [25,50,100,250,500],
'max_depth' : [10,25,50,125,250],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 3:")

grid_arc3 = GridSearchCV(RFC_arc3 ,params_arc3, scoring='accuracy',cv=5) #43 min
grid_arc3.fit(X_arc3_train,y_arc3_train)

print(grid_arc3.best_params_, grid_arc3.best_score_)
rf_best_arc3 = grid_arc3.best_estimator_

y_arc3_pred = rf_best_arc3.predict(X_arc3_test)

accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

print(rf_best_arc3.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc3_train.columns,
    "Imp": rf_best_arc3.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc3

In [ ]:
Y_arc3 = LabelEncoder().fit_transform(Y_arc3)

Y_arc3

In [ ]:
X_arc3_train, X_arc3_test, y_arc3_train, y_arc3_test = train_test_split(X_arc3, Y_arc3, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc3 = SVC(kernel = 'linear', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc3 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc3 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc3 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc3_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc3_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc3_SVC_Linear,params_arc3_SVC_RS]:
  print("Otimização Dados Arquitetura 3:")
  grid_arc3_SVC = GridSearchCV(SVC_arc3 ,params, scoring='accuracy')
  grid_arc3_SVC.fit(X_arc3_train,y_arc3_train)
  print(grid_arc3_SVC.best_params_, grid_arc3_SVC.best_score_)
  rf_best_arc3_SVC = grid_arc3_SVC.best_estimator_

  y_arc3_pred_SVC = rf_best_arc3_SVC.predict(X_arc3_test)

  accuracy_score(y_arc3_test, y_arc3_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc3

In [ ]:
df_arc3.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc3

In [ ]:
df_arc3.to_csv('DataframeML_ARC3_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc3=list(df_arc3)
print(coln_arc3, "\n")

print(len(coln_arc3))
#Definindo as Features (X's):
features=coln_arc3[1:]
X_arc3=df_arc3[features]

print(coln_arc3[1:])
#Definindo os Targets (Y's):

targets=coln_arc3[0] #Definindo lista de targets
Y_arc3=df_arc3[targets]

In [ ]:
Y_arc3

In [ ]:
X_arc3

## Random Forest (SEM TEMPO)

In [ ]:
X_arc3

In [ ]:
Y_arc3 = LabelEncoder().fit_transform(Y_arc3)

Y_arc3

In [ ]:
X_arc3_train, X_arc3_test, y_arc3_train, y_arc3_test = train_test_split(X_arc3, Y_arc3, test_size = 0.3, random_state = 0)

RFC_arc3 = RandomForestClassifier(max_depth= 100, n_estimators = 100, random_state = 0)
RFC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = RFC_arc3.predict(X_arc3_test)

In [ ]:
train_score_rf=RFC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
RFC_arc3.feature_importances_

In [ ]:
np.max(RFC_arc3.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc3_train.columns,
    "Imp": RFC_arc3.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,RFC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,RFC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc3 = {
'n_estimators': [25,50,100,250,500],
'max_depth' : [10,25,50,125,250],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 3:")

grid_arc3 = GridSearchCV(RFC_arc3 ,params_arc3, scoring='accuracy',cv=5) #43 min
grid_arc3.fit(X_arc3_train,y_arc3_train)

print(grid_arc3.best_params_, grid_arc3.best_score_)
rf_best_arc3 = grid_arc3.best_estimator_

y_arc3_pred = rf_best_arc3.predict(X_arc3_test)

accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

print(rf_best_arc3.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc3_train.columns,
    "Imp": rf_best_arc3.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc3

In [ ]:
Y_arc3 = LabelEncoder().fit_transform(Y_arc3)

Y_arc3

In [ ]:
X_arc3_train, X_arc3_test, y_arc3_train, y_arc3_test = train_test_split(X_arc3, Y_arc3, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc3 = SVC(kernel = 'linear', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc3 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc3 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc3 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc3.fit(X_arc3_train, y_arc3_train)

In [ ]:
y_arc3_pred = SVC_arc3.predict(X_arc3_test)

In [ ]:
train_score_svc=SVC_arc3.score(X_arc3_train,y_arc3_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc3.score(X_arc3_test,y_arc3_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc3_test, y_arc3_pred)
print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc3_train,SVC_arc3.predict(X_arc3_train))
confmatrix_test=confusion_matrix(y_arc3_test,SVC_arc3.predict(X_arc3_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc3_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc3_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc3_SVC_Linear,params_arc3_SVC_RS]:
  print("Otimização Dados Arquitetura 3 - Sem tempo:")
  grid_arc3_SVC = GridSearchCV(SVC_arc3 ,params, scoring='accuracy')
  grid_arc3_SVC.fit(X_arc3_train,y_arc3_train)
  print(grid_arc3_SVC.best_params_, grid_arc3_SVC.best_score_)
  rf_best_arc3_SVC = grid_arc3_SVC.best_estimator_

  y_arc3_pred_SVC = rf_best_arc3_SVC.predict(X_arc3_test)

  accuracy_score(y_arc3_test, y_arc3_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc3_test, y_arc3_pred_SVC)*100)
  print()

# Arquitetura 4 :
PCA (0.70)-> 71 Colunas

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.70) # 70% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc4=pd.concat(frames,axis=1)

In [ ]:
df_arc4

In [ ]:
df_arc4.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc4.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc4.to_csv('DataframeML_ARC4_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc4=list(df_arc4)
print(coln_arc4, "\n")

print(len(coln_arc4))
#Definindo as Features (X's):
features=coln_arc4[1:]
X_arc4=df_arc4[features]

print(coln_arc4[1:])
#Definindo os Targets (Y's):

targets=coln_arc4[0] #Definindo lista de targets
Y_arc4=df_arc4[targets]

In [ ]:
X_arc4

In [ ]:
Y_arc4

## Random Forest (COM TEMPO)

In [ ]:
X_arc4

In [ ]:
Y_arc4 = LabelEncoder().fit_transform(Y_arc4)

Y_arc4

In [ ]:
X_arc4_train, X_arc4_test, y_arc4_train, y_arc4_test = train_test_split(X_arc4, Y_arc4, test_size = 0.3, random_state = 0)

RFC_arc4 = RandomForestClassifier(max_depth= 25, n_estimators = 50, random_state = 0)
RFC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = RFC_arc4.predict(X_arc4_test)

In [ ]:
train_score_rf=RFC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
RFC_arc4.feature_importances_

In [ ]:
np.max(RFC_arc4.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc4_train.columns,
    "Imp": RFC_arc4.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,RFC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,RFC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc4 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 4:")

grid_arc4 = GridSearchCV(RFC_arc4 ,params_arc4, scoring='accuracy',cv=5) #43 min
grid_arc4.fit(X_arc4_train,y_arc4_train)

print(grid_arc4.best_params_, grid_arc4.best_score_)
rf_best_arc4 = grid_arc4.best_estimator_

y_arc4_pred = rf_best_arc4.predict(X_arc4_test)

accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

print(rf_best_arc4.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc4_train.columns,
    "Imp": rf_best_arc4.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc4

In [ ]:
Y_arc4 = LabelEncoder().fit_transform(Y_arc4)

Y_arc4

In [ ]:
X_arc4_train, X_arc4_test, y_arc4_train, y_arc4_test = train_test_split(X_arc4, Y_arc4, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc4 = SVC(kernel = 'linear', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc4 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc4 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc4 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc4_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc4_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc4_SVC_Linear,params_arc4_SVC_RS]:
  print("Otimização Dados Arquitetura 4:")
  grid_arc4_SVC = GridSearchCV(SVC_arc4 ,params, scoring='accuracy')
  grid_arc4_SVC.fit(X_arc4_train,y_arc4_train)
  print(grid_arc4_SVC.best_params_, grid_arc4_SVC.best_score_)
  rf_best_arc4_SVC = grid_arc4_SVC.best_estimator_

  y_arc4_pred_SVC = rf_best_arc4_SVC.predict(X_arc4_test)

  accuracy_score(y_arc4_test, y_arc4_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc4

In [ ]:
df_arc4.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc4

In [ ]:
df_arc4.to_csv('DataframeML_ARC4_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc4=list(df_arc4)
print(coln_arc4, "\n")

print(len(coln_arc4))
#Definindo as Features (X's):
features=coln_arc4[1:]
X_arc4=df_arc4[features]

print(coln_arc4[1:])
#Definindo os Targets (Y's):

targets=coln_arc4[0] #Definindo lista de targets
Y_arc4=df_arc4[targets]

In [ ]:
Y_arc4

In [ ]:
X_arc4

## Random Forest (SEM TEMPO)

In [ ]:
X_arc4

In [ ]:
Y_arc4 = LabelEncoder().fit_transform(Y_arc4)

Y_arc4

In [ ]:
X_arc4_train, X_arc4_test, y_arc4_train, y_arc4_test = train_test_split(X_arc4, Y_arc4, test_size = 0.3, random_state = 0)

RFC_arc4 = RandomForestClassifier(max_depth= 10, n_estimators = 25, random_state = 0)
RFC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = RFC_arc4.predict(X_arc4_test)

In [ ]:
train_score_rf=RFC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
RFC_arc4.feature_importances_

In [ ]:
np.max(RFC_arc4.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc4_train.columns,
    "Imp": RFC_arc4.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,RFC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,RFC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc4 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 4:")

grid_arc4 = GridSearchCV(RFC_arc4 ,params_arc4, scoring='accuracy',cv=5) #43 min
grid_arc4.fit(X_arc4_train,y_arc4_train)

print(grid_arc4.best_params_, grid_arc4.best_score_)
rf_best_arc4 = grid_arc4.best_estimator_

y_arc4_pred = rf_best_arc4.predict(X_arc4_test)

accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

print(rf_best_arc4.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc4_train.columns,
    "Imp": rf_best_arc4.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc4

In [ ]:
Y_arc4 = LabelEncoder().fit_transform(Y_arc4)

Y_arc4

In [ ]:
X_arc4_train, X_arc4_test, y_arc4_train, y_arc4_test = train_test_split(X_arc4, Y_arc4, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc4 = SVC(kernel = 'linear', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc4 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc4 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc4 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc4.fit(X_arc4_train, y_arc4_train)

In [ ]:
y_arc4_pred = SVC_arc4.predict(X_arc4_test)

In [ ]:
train_score_svc=SVC_arc4.score(X_arc4_train,y_arc4_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc4.score(X_arc4_test,y_arc4_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc4_test, y_arc4_pred)
print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc4_train,SVC_arc4.predict(X_arc4_train))
confmatrix_test=confusion_matrix(y_arc4_test,SVC_arc4.predict(X_arc4_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc4_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc4_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc4_SVC_Linear,params_arc4_SVC_RS]:
  print("Otimização Dados Arquitetura 4:")
  grid_arc4_SVC = GridSearchCV(SVC_arc4 ,params, scoring='accuracy')
  grid_arc4_SVC.fit(X_arc4_train,y_arc4_train)
  print(grid_arc4_SVC.best_params_, grid_arc4_SVC.best_score_)
  rf_best_arc4_SVC = grid_arc4_SVC.best_estimator_

  y_arc4_pred_SVC = rf_best_arc4_SVC.predict(X_arc4_test)

  accuracy_score(y_arc4_test, y_arc4_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc4_test, y_arc4_pred_SVC)*100)
  print()

# Arquitetura 5

PCA(0.50)

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.50) # 70% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc5=pd.concat(frames,axis=1)

In [ ]:
df_arc5

In [ ]:
df_arc5.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc5.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc5.to_csv('DataframeML_ARC5_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc5=list(df_arc5)
print(coln_arc5, "\n")

print(len(coln_arc5))
#Definindo as Features (X's):
features=coln_arc5[1:]
X_arc5=df_arc5[features]

print(coln_arc5[1:])
#Definindo os Targets (Y's):

targets=coln_arc5[0] #Definindo lista de targets
Y_arc5=df_arc5[targets]

In [ ]:
X_arc5

In [ ]:
Y_arc5

## Random Forest (COM TEMPO)

In [ ]:
X_arc5

In [ ]:
Y_arc5 = LabelEncoder().fit_transform(Y_arc5)

Y_arc5

In [ ]:
X_arc5_train, X_arc5_test, y_arc5_train, y_arc5_test = train_test_split(X_arc5, Y_arc5, test_size = 0.3, random_state = 0)

RFC_arc5 = RandomForestClassifier(max_depth= 25, n_estimators = 50, random_state = 0)
RFC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = RFC_arc5.predict(X_arc5_test)

In [ ]:
train_score_rf=RFC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
RFC_arc5.feature_importances_

In [ ]:
np.max(RFC_arc5.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc5_train.columns,
    "Imp": RFC_arc5.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,RFC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,RFC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc5 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 5:")

grid_arc5 = GridSearchCV(RFC_arc5 ,params_arc5, scoring='accuracy',cv=5)
grid_arc5.fit(X_arc5_train,y_arc5_train)

print(grid_arc5.best_params_, grid_arc5.best_score_)
rf_best_arc5 = grid_arc5.best_estimator_

y_arc5_pred = rf_best_arc5.predict(X_arc5_test)

accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

print(rf_best_arc5.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc5_train.columns,
    "Imp": rf_best_arc5.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc5

In [ ]:
Y_arc5 = LabelEncoder().fit_transform(Y_arc5)

Y_arc5

In [ ]:
X_arc5_train, X_arc5_test, y_arc5_train, y_arc5_test = train_test_split(X_arc5, Y_arc5, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc5 = SVC(kernel = 'linear', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc5 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc5 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc5 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc5_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc5_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc5_SVC_Linear,params_arc5_SVC_RS]:
  print("Otimização Dados Arquitetura 5:")
  grid_arc5_SVC = GridSearchCV(SVC_arc5 ,params, scoring='accuracy')
  grid_arc5_SVC.fit(X_arc5_train,y_arc5_train)
  print(grid_arc5_SVC.best_params_, grid_arc5_SVC.best_score_)
  rf_best_arc5_SVC = grid_arc5_SVC.best_estimator_

  y_arc5_pred_SVC = rf_best_arc5_SVC.predict(X_arc5_test)

  accuracy_score(y_arc5_test, y_arc5_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc5

In [ ]:
df_arc5.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc5

In [ ]:
df_arc5.to_csv('DataframeML_ARC5_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc5=list(df_arc5)
print(coln_arc5, "\n")

print(len(coln_arc5))
#Definindo as Features (X's):
features=coln_arc5[1:]
X_arc5=df_arc5[features]

print(coln_arc5[1:])
#Definindo os Targets (Y's):

targets=coln_arc5[0] #Definindo lista de targets
Y_arc5=df_arc5[targets]

In [ ]:
Y_arc5

In [ ]:
X_arc5

## Random Forest (SEM TEMPO)

In [ ]:
X_arc5

In [ ]:
Y_arc5 = LabelEncoder().fit_transform(Y_arc5)

Y_arc5

In [ ]:
X_arc5_train, X_arc5_test, y_arc5_train, y_arc5_test = train_test_split(X_arc5, Y_arc5, test_size = 0.3, random_state = 0)

RFC_arc5 = RandomForestClassifier(max_depth= 10, n_estimators = 25, random_state = 0)
RFC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = RFC_arc5.predict(X_arc5_test)

In [ ]:
train_score_rf=RFC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
RFC_arc5.feature_importances_

In [ ]:
np.max(RFC_arc5.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc5_train.columns,
    "Imp": RFC_arc5.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,RFC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,RFC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc5 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 5:")

grid_arc5 = GridSearchCV(RFC_arc5 ,params_arc5, scoring='accuracy',cv=5)
grid_arc5.fit(X_arc5_train,y_arc5_train)

print(grid_arc5.best_params_, grid_arc5.best_score_)
rf_best_arc5 = grid_arc5.best_estimator_

y_arc5_pred = rf_best_arc5.predict(X_arc5_test)

accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

print(rf_best_arc5.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc5_train.columns,
    "Imp": rf_best_arc5.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc5

In [ ]:
Y_arc5 = LabelEncoder().fit_transform(Y_arc5)

Y_arc5

In [ ]:
X_arc5_train, X_arc5_test, y_arc5_train, y_arc5_test = train_test_split(X_arc5, Y_arc5, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc5 = SVC(kernel = 'linear', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc5 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc5 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc5 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc5.fit(X_arc5_train, y_arc5_train)

In [ ]:
y_arc5_pred = SVC_arc5.predict(X_arc5_test)

In [ ]:
train_score_svc=SVC_arc5.score(X_arc5_train,y_arc5_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc5.score(X_arc5_test,y_arc5_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc5_test, y_arc5_pred)
print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc5_train,SVC_arc5.predict(X_arc5_train))
confmatrix_test=confusion_matrix(y_arc5_test,SVC_arc5.predict(X_arc5_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc5_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc5_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc5_SVC_Linear,params_arc5_SVC_RS]:
  print("Otimização Dados Arquitetura 5:")
  grid_arc5_SVC = GridSearchCV(SVC_arc5 ,params, scoring='accuracy')
  grid_arc5_SVC.fit(X_arc5_train,y_arc5_train)
  print(grid_arc5_SVC.best_params_, grid_arc5_SVC.best_score_)
  rf_best_arc5_SVC = grid_arc5_SVC.best_estimator_

  y_arc5_pred_SVC = rf_best_arc5_SVC.predict(X_arc5_test)

  accuracy_score(y_arc5_test, y_arc5_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc5_test, y_arc5_pred_SVC)*100)
  print()

# Arquitetura 6

PCA(0.30)

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.30) # 30% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc6=pd.concat(frames,axis=1)

In [ ]:
df_arc6

In [ ]:
df_arc6.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc6.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc6.to_csv('DataframeML_ARC6_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc6=list(df_arc6)
print(coln_arc6, "\n")

print(len(coln_arc6))
#Definindo as Features (X's):
features=coln_arc6[1:]
X_arc6=df_arc6[features]

print(coln_arc6[1:])
#Definindo os Targets (Y's):

targets=coln_arc6[0] #Definindo lista de targets
Y_arc6=df_arc6[targets]

In [ ]:
X_arc6

In [ ]:
Y_arc6

## Random Forest (COM TEMPO)

In [ ]:
X_arc6

In [ ]:
Y_arc6 = LabelEncoder().fit_transform(Y_arc6)

Y_arc6

In [ ]:
X_arc6_train, X_arc6_test, y_arc6_train, y_arc6_test = train_test_split(X_arc6, Y_arc6, test_size = 0.3, random_state = 0)

RFC_arc6 = RandomForestClassifier(max_depth= 25, n_estimators = 50, random_state = 0)
RFC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = RFC_arc6.predict(X_arc6_test)

In [ ]:
train_score_rf=RFC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
RFC_arc6.feature_importances_

In [ ]:
np.max(RFC_arc6.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc6_train.columns,
    "Imp": RFC_arc6.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,RFC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,RFC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc6 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 6:")

grid_arc6 = GridSearchCV(RFC_arc6 ,params_arc6, scoring='accuracy',cv=5)
grid_arc6.fit(X_arc6_train,y_arc6_train)

print(grid_arc6.best_params_, grid_arc6.best_score_)
rf_best_arc6 = grid_arc6.best_estimator_

y_arc6_pred = rf_best_arc6.predict(X_arc6_test)

accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

print(rf_best_arc6.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc6_train.columns,
    "Imp": rf_best_arc6.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc6

In [ ]:
Y_arc6 = LabelEncoder().fit_transform(Y_arc6)

Y_arc6

In [ ]:
X_arc6_train, X_arc6_test, y_arc6_train, y_arc6_test = train_test_split(X_arc6, Y_arc6, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc6 = SVC(kernel = 'linear', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc6 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc6 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc6 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc6_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc6_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc6_SVC_Linear,params_arc6_SVC_RS]:
  print("Otimização Dados Arquitetura 6:")
  grid_arc6_SVC = GridSearchCV(SVC_arc6 ,params, scoring='accuracy')
  grid_arc6_SVC.fit(X_arc6_train,y_arc6_train)
  print(grid_arc6_SVC.best_params_, grid_arc6_SVC.best_score_)
  rf_best_arc6_SVC = grid_arc6_SVC.best_estimator_

  y_arc6_pred_SVC = rf_best_arc6_SVC.predict(X_arc6_test)

  accuracy_score(y_arc6_test, y_arc6_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc6

In [ ]:
df_arc6.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc6

In [ ]:
df_arc6.to_csv('DataframeML_ARC6_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc6=list(df_arc6)
print(coln_arc6, "\n")

print(len(coln_arc6))
#Definindo as Features (X's):
features=coln_arc6[1:]
X_arc6=df_arc6[features]

print(coln_arc6[1:])
#Definindo os Targets (Y's):

targets=coln_arc6[0] #Definindo lista de targets
Y_arc6=df_arc6[targets]

In [ ]:
Y_arc6

In [ ]:
X_arc6

## Random Forest (SEM TEMPO)

In [ ]:
X_arc6

In [ ]:
Y_arc6 = LabelEncoder().fit_transform(Y_arc6)

Y_arc6

In [ ]:
X_arc6_train, X_arc6_test, y_arc6_train, y_arc6_test = train_test_split(X_arc6, Y_arc6, test_size = 0.3, random_state = 0)

RFC_arc6 = RandomForestClassifier(max_depth= 10, n_estimators = 25, random_state = 0)
RFC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = RFC_arc6.predict(X_arc6_test)

In [ ]:
train_score_rf=RFC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
RFC_arc6.feature_importances_

In [ ]:
np.max(RFC_arc6.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc6_train.columns,
    "Imp": RFC_arc6.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,RFC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,RFC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc6 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 6:")

grid_arc6 = GridSearchCV(RFC_arc6 ,params_arc6, scoring='accuracy',cv=5) #43 min
grid_arc6.fit(X_arc6_train,y_arc6_train)

print(grid_arc6.best_params_, grid_arc6.best_score_)
rf_best_arc6 = grid_arc6.best_estimator_

y_arc6_pred = rf_best_arc6.predict(X_arc6_test)

accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

print(rf_best_arc6.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc6_train.columns,
    "Imp": rf_best_arc6.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc6

In [ ]:
Y_arc6 = LabelEncoder().fit_transform(Y_arc6)

Y_arc6

In [ ]:
X_arc6_train, X_arc6_test, y_arc6_train, y_arc6_test = train_test_split(X_arc6, Y_arc6, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc6 = SVC(kernel = 'linear', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc6 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc6 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc6 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc6.fit(X_arc6_train, y_arc6_train)

In [ ]:
y_arc6_pred = SVC_arc6.predict(X_arc6_test)

In [ ]:
train_score_svc=SVC_arc6.score(X_arc6_train,y_arc6_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc6.score(X_arc6_test,y_arc6_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc6_test, y_arc6_pred)
print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc6_train,SVC_arc6.predict(X_arc6_train))
confmatrix_test=confusion_matrix(y_arc6_test,SVC_arc6.predict(X_arc6_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc6_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc6_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc6_SVC_Linear,params_arc6_SVC_RS]:
  print("Otimização Dados Arquitetura 6:")
  grid_arc6_SVC = GridSearchCV(SVC_arc6 ,params, scoring='accuracy')
  grid_arc6_SVC.fit(X_arc6_train,y_arc6_train)
  print(grid_arc6_SVC.best_params_, grid_arc6_SVC.best_score_)
  rf_best_arc6_SVC = grid_arc6_SVC.best_estimator_

  y_arc6_pred_SVC = rf_best_arc6_SVC.predict(X_arc6_test)

  accuracy_score(y_arc6_test, y_arc6_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc6_test, y_arc6_pred_SVC)*100)
  print()

# Arquitetura 7

PCA(0.20)

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.20) # 20% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc7=pd.concat(frames,axis=1)

In [ ]:
df_arc7

In [ ]:
df_arc7.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc7.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc7.to_csv('DataframeML_ARC7_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc7=list(df_arc7)
print(coln_arc7, "\n")

print(len(coln_arc7))
#Definindo as Features (X's):
features=coln_arc7[1:]
X_arc7=df_arc7[features]

print(coln_arc7[1:])
#Definindo os Targets (Y's):

targets=coln_arc7[0] #Definindo lista de targets
Y_arc7=df_arc7[targets]

In [ ]:
X_arc7

In [ ]:
Y_arc7

## Random Forest (COM TEMPO)

In [ ]:
X_arc7

In [ ]:
Y_arc7 = LabelEncoder().fit_transform(Y_arc7)

Y_arc7

In [ ]:
X_arc7_train, X_arc7_test, y_arc7_train, y_arc7_test = train_test_split(X_arc7, Y_arc7, test_size = 0.3, random_state = 0)

RFC_arc7 = RandomForestClassifier(max_depth= 25, n_estimators = 50, random_state = 0)
RFC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = RFC_arc7.predict(X_arc7_test)

In [ ]:
train_score_rf=RFC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
RFC_arc7.feature_importances_

In [ ]:
np.max(RFC_arc7.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc7_train.columns,
    "Imp": RFC_arc7.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,RFC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,RFC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc7 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 7:")

grid_arc7 = GridSearchCV(RFC_arc7 ,params_arc7, scoring='accuracy',cv=5)
grid_arc7.fit(X_arc7_train,y_arc7_train)

print(grid_arc7.best_params_, grid_arc7.best_score_)
rf_best_arc7 = grid_arc7.best_estimator_

y_arc7_pred = rf_best_arc7.predict(X_arc7_test)

accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

print(rf_best_arc7.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc7_train.columns,
    "Imp": rf_best_arc7.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc7

In [ ]:
Y_arc7 = LabelEncoder().fit_transform(Y_arc7)

Y_arc7

In [ ]:
X_arc7_train, X_arc7_test, y_arc7_train, y_arc7_test = train_test_split(X_arc7, Y_arc7, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc7 = SVC(kernel = 'linear', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc7 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc7 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc7 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc7_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc7_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc7_SVC_Linear,params_arc7_SVC_RS]:
  print("Otimização Dados Arquitetura 7:")
  grid_arc7_SVC = GridSearchCV(SVC_arc7 ,params, scoring='accuracy')
  grid_arc7_SVC.fit(X_arc7_train,y_arc7_train)
  print(grid_arc7_SVC.best_params_, grid_arc7_SVC.best_score_)
  rf_best_arc7_SVC = grid_arc7_SVC.best_estimator_

  y_arc7_pred_SVC = rf_best_arc7_SVC.predict(X_arc7_test)

  accuracy_score(y_arc7_test, y_arc7_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc7

In [ ]:
df_arc7.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc7

In [ ]:
df_arc7.to_csv('DataframeML_ARC7_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc7=list(df_arc7)
print(coln_arc7, "\n")

print(len(coln_arc7))
#Definindo as Features (X's):
features=coln_arc7[1:]
X_arc7=df_arc7[features]

print(coln_arc7[1:])
#Definindo os Targets (Y's):

targets=coln_arc7[0] #Definindo lista de targets
Y_arc7=df_arc7[targets]

In [ ]:
Y_arc7

In [ ]:
X_arc7

## Random Forest (SEM TEMPO)

In [ ]:
X_arc7

In [ ]:
Y_arc7 = LabelEncoder().fit_transform(Y_arc7)

Y_arc7

In [ ]:
X_arc7_train, X_arc7_test, y_arc7_train, y_arc7_test = train_test_split(X_arc7, Y_arc7, test_size = 0.3, random_state = 0)

RFC_arc7 = RandomForestClassifier(max_depth= 10, n_estimators = 25, random_state = 0)
RFC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = RFC_arc7.predict(X_arc7_test)

In [ ]:
train_score_rf=RFC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
RFC_arc7.feature_importances_

In [ ]:
np.max(RFC_arc7.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc7_train.columns,
    "Imp": RFC_arc7.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,RFC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,RFC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc7 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 7:")

grid_arc7 = GridSearchCV(RFC_arc7 ,params_arc7, scoring='accuracy',cv=5) #43 min
grid_arc7.fit(X_arc7_train,y_arc7_train)

print(grid_arc7.best_params_, grid_arc7.best_score_)
rf_best_arc7 = grid_arc7.best_estimator_

y_arc7_pred = rf_best_arc7.predict(X_arc7_test)

accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

print(rf_best_arc7.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc7_train.columns,
    "Imp": rf_best_arc7.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc7

In [ ]:
Y_arc7 = LabelEncoder().fit_transform(Y_arc7)

Y_arc7

In [ ]:
X_arc7_train, X_arc7_test, y_arc7_train, y_arc7_test = train_test_split(X_arc7, Y_arc7, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc7 = SVC(kernel = 'linear', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc7 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc7 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc7 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc7.fit(X_arc7_train, y_arc7_train)

In [ ]:
y_arc7_pred = SVC_arc7.predict(X_arc7_test)

In [ ]:
train_score_svc=SVC_arc7.score(X_arc7_train,y_arc7_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc7.score(X_arc7_test,y_arc7_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc7_test, y_arc7_pred)
print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc7_train,SVC_arc7.predict(X_arc7_train))
confmatrix_test=confusion_matrix(y_arc7_test,SVC_arc7.predict(X_arc7_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc7_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc7_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc7_SVC_Linear,params_arc7_SVC_RS]:
  print("Otimização Dados Arquitetura 7:")
  grid_arc7_SVC = GridSearchCV(SVC_arc7 ,params, scoring='accuracy')
  grid_arc7_SVC.fit(X_arc7_train,y_arc7_train)
  print(grid_arc7_SVC.best_params_, grid_arc7_SVC.best_score_)
  rf_best_arc7_SVC = grid_arc7_SVC.best_estimator_

  y_arc7_pred_SVC = rf_best_arc7_SVC.predict(X_arc7_test)

  accuracy_score(y_arc7_test, y_arc7_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc7_test, y_arc7_pred_SVC)*100)
  print()

# Arquitetura 8

PCA(0.10)

## Montando Nova Tabela

* Zeramos as tranformações, voltamos ao início

In [ ]:
#Definindo as Features (X's):
features=coln[1:]
X=df_ML[features]

#Definindo os Targets (Y's):

targets=coln[0] #Definindo lista de targets
Y=df_ML[targets]

In [ ]:
#Dividindo dados no Tempo
X_tempo = X[c_T]
c_X_tempo=list(X_tempo)
print(c_X_tempo)

#Dividindo as FFT's por Segmentos / Sensores / Eixos (SUBDATASETS):
X_fft_a1_x=X[c_F_a1_x]
X_fft_a1_y=X[c_F_a1_y]
X_fft_a1_z=X[c_F_a1_z]

X_fft_a2_x=X[c_F_a2_x]
X_fft_a2_y=X[c_F_a2_y]
X_fft_a2_z=X[c_F_a2_z]

X_fft_a3_x=X[c_F_a3_x]
X_fft_a3_y=X[c_F_a3_y]
X_fft_a3_z=X[c_F_a3_z]

X_fft_g1_x=X[c_F_g1_x]
X_fft_g1_y=X[c_F_g1_y]
X_fft_g1_z=X[c_F_g1_z]

X_fft_g2_x=X[c_F_g2_x]
X_fft_g2_y=X[c_F_g2_y]
X_fft_g2_z=X[c_F_g2_z]

X_fft_g3_x=X[c_F_g3_x]
X_fft_g3_y=X[c_F_g3_y]
X_fft_g3_z=X[c_F_g3_z]

In [ ]:
sc = StandardScaler()

#Aplicando a cada X_fft e Tempo:

X_tempo = sc.fit_transform(X_tempo)

X_fft_a1_x = sc.fit_transform(X_fft_a1_x)
X_fft_a1_y = sc.fit_transform(X_fft_a1_y)
X_fft_a1_z = sc.fit_transform(X_fft_a1_z)

X_fft_a2_x = sc.fit_transform(X_fft_a2_x)
X_fft_a2_y = sc.fit_transform(X_fft_a2_y)
X_fft_a2_z = sc.fit_transform(X_fft_a2_z)

X_fft_a3_x = sc.fit_transform(X_fft_a3_x)
X_fft_a3_y = sc.fit_transform(X_fft_a3_y)
X_fft_a3_z = sc.fit_transform(X_fft_a3_z)

X_fft_g1_x = sc.fit_transform(X_fft_g1_x)
X_fft_g1_y = sc.fit_transform(X_fft_g1_y)
X_fft_g1_z = sc.fit_transform(X_fft_g1_z)

X_fft_g2_x = sc.fit_transform(X_fft_g2_x)
X_fft_g2_y = sc.fit_transform(X_fft_g2_y)
X_fft_g2_z = sc.fit_transform(X_fft_g2_z)

X_fft_g3_x = sc.fit_transform(X_fft_g3_x)
X_fft_g3_y = sc.fit_transform(X_fft_g3_y)
X_fft_g3_z = sc.fit_transform(X_fft_g3_z)

In [ ]:
df_Y=pd.DataFrame(Y, columns = ['Classificação'])
df_Y

In [ ]:
df_X_tempo=pd.DataFrame(X_tempo, columns = c_X_tempo)

df_X_tempo

In [ ]:
pca = PCA(0.10) # 10% de Variância Explicada

In [ ]:
#Aplicando o PCA para cada FFT:

PCA_X_fft_a1_x = pca.fit_transform(X_fft_a1_x)
PCA_X_fft_a1_y = pca.fit_transform(X_fft_a1_y)
PCA_X_fft_a1_z = pca.fit_transform(X_fft_a1_z)

PCA_X_fft_a2_x = pca.fit_transform(X_fft_a2_x)
PCA_X_fft_a2_y = pca.fit_transform(X_fft_a2_y)
PCA_X_fft_a2_z = pca.fit_transform(X_fft_a2_z)

PCA_X_fft_a3_x = pca.fit_transform(X_fft_a3_x)
PCA_X_fft_a3_y = pca.fit_transform(X_fft_a3_y)
PCA_X_fft_a3_z = pca.fit_transform(X_fft_a3_z)

PCA_X_fft_g1_x = pca.fit_transform(X_fft_g1_x)
PCA_X_fft_g1_y = pca.fit_transform(X_fft_g1_y)
PCA_X_fft_g1_z = pca.fit_transform(X_fft_g1_z)

PCA_X_fft_g2_x = pca.fit_transform(X_fft_g2_x)
PCA_X_fft_g2_y = pca.fit_transform(X_fft_g2_y)
PCA_X_fft_g2_z = pca.fit_transform(X_fft_g2_z)

PCA_X_fft_g3_x = pca.fit_transform(X_fft_g3_x)
PCA_X_fft_g3_y = pca.fit_transform(X_fft_g3_y)
PCA_X_fft_g3_z = pca.fit_transform(X_fft_g3_z)

In [ ]:
#Sub DataFrames:
df_PCA_FFT_a1_X = pd.DataFrame(PCA_X_fft_a1_x, columns = ['PCA_FFT_a1_X_'+str(i) for i in range(len(PCA_X_fft_a1_x[0]))])
df_PCA_FFT_a1_Y = pd.DataFrame(PCA_X_fft_a1_y, columns = ['PCA_FFT_a1_Y_'+str(i) for i in range(len(PCA_X_fft_a1_y[0]))])
df_PCA_FFT_a1_Z = pd.DataFrame(PCA_X_fft_a1_z, columns = ['PCA_FFT_a1_Z_'+str(i) for i in range(len(PCA_X_fft_a1_z[0]))])

df_PCA_FFT_a2_X = pd.DataFrame(PCA_X_fft_a2_x, columns = ['PCA_FFT_a2_X_'+str(i) for i in range(len(PCA_X_fft_a2_x[0]))])
df_PCA_FFT_a2_Y = pd.DataFrame(PCA_X_fft_a2_y, columns = ['PCA_FFT_a2_Y_'+str(i) for i in range(len(PCA_X_fft_a2_y[0]))])
df_PCA_FFT_a2_Z = pd.DataFrame(PCA_X_fft_a2_z, columns = ['PCA_FFT_a2_Z_'+str(i) for i in range(len(PCA_X_fft_a2_z[0]))])

df_PCA_FFT_a3_X = pd.DataFrame(PCA_X_fft_a3_x, columns = ['PCA_FFT_a3_X_'+str(i) for i in range(len(PCA_X_fft_a3_x[0]))])
df_PCA_FFT_a3_Y = pd.DataFrame(PCA_X_fft_a3_y, columns = ['PCA_FFT_a3_Y_'+str(i) for i in range(len(PCA_X_fft_a3_y[0]))])
df_PCA_FFT_a3_Z = pd.DataFrame(PCA_X_fft_a3_z, columns = ['PCA_FFT_a3_Z_'+str(i) for i in range(len(PCA_X_fft_a3_z[0]))])

df_PCA_FFT_g1_X = pd.DataFrame(PCA_X_fft_g1_x, columns = ['PCA_FFT_g1_X_'+str(i) for i in range(len(PCA_X_fft_g1_x[0]))])
df_PCA_FFT_g1_Y = pd.DataFrame(PCA_X_fft_g1_y, columns = ['PCA_FFT_g1_Y_'+str(i) for i in range(len(PCA_X_fft_g1_y[0]))])
df_PCA_FFT_g1_Z = pd.DataFrame(PCA_X_fft_g1_z, columns = ['PCA_FFT_g1_Z_'+str(i) for i in range(len(PCA_X_fft_g1_z[0]))])

df_PCA_FFT_g2_X = pd.DataFrame(PCA_X_fft_g2_x, columns = ['PCA_FFT_g2_X_'+str(i) for i in range(len(PCA_X_fft_g2_x[0]))])
df_PCA_FFT_g2_Y = pd.DataFrame(PCA_X_fft_g2_y, columns = ['PCA_FFT_g2_Y_'+str(i) for i in range(len(PCA_X_fft_g2_y[0]))])
df_PCA_FFT_g2_Z = pd.DataFrame(PCA_X_fft_g2_z, columns = ['PCA_FFT_g2_Z_'+str(i) for i in range(len(PCA_X_fft_g2_z[0]))])

df_PCA_FFT_g3_X = pd.DataFrame(PCA_X_fft_g3_x, columns = ['PCA_FFT_g3_X_'+str(i) for i in range(len(PCA_X_fft_g3_x[0]))])
df_PCA_FFT_g3_Y = pd.DataFrame(PCA_X_fft_g3_y, columns = ['PCA_FFT_g3_Y_'+str(i) for i in range(len(PCA_X_fft_g3_y[0]))])
df_PCA_FFT_g3_Z = pd.DataFrame(PCA_X_fft_g3_z, columns = ['PCA_FFT_g3_Z_'+str(i) for i in range(len(PCA_X_fft_g3_z[0]))])

In [ ]:
#Teste:
df_PCA_FFT_a3_Z

In [ ]:
#Agrupando Tudo e Montar Tabela para Arquitetura 1:
frames=[df_Y,df_X_tempo,df_PCA_FFT_a1_X,df_PCA_FFT_a1_Y,df_PCA_FFT_a1_Z,df_PCA_FFT_a2_X,df_PCA_FFT_a2_Y,df_PCA_FFT_a2_Z,df_PCA_FFT_a3_X,df_PCA_FFT_a3_Y,df_PCA_FFT_a3_Z,df_PCA_FFT_g1_X,df_PCA_FFT_g1_Y,df_PCA_FFT_g1_Z,df_PCA_FFT_g2_X,df_PCA_FFT_g2_Y,df_PCA_FFT_g2_Z,df_PCA_FFT_g3_X,df_PCA_FFT_g3_Y,df_PCA_FFT_g3_Z]
df_arc8=pd.concat(frames,axis=1)

In [ ]:
df_arc8

In [ ]:
df_arc8.isnull().sum()

In [ ]:
#Conferindo NANs:
for x in df_arc8.isnull().sum():
  if x != 0:
    print('TEM ALGUM NAN')
print("Pode Seguir !!! Sem NANs")

In [ ]:
df_arc8.to_csv('DataframeML_ARC8_TempoFFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc8=list(df_arc8)
print(coln_arc8, "\n")

print(len(coln_arc8))
#Definindo as Features (X's):
features=coln_arc8[1:]
X_arc8=df_arc8[features]

print(coln_arc8[1:])
#Definindo os Targets (Y's):

targets=coln_arc8[0] #Definindo lista de targets
Y_arc8=df_arc8[targets]

In [ ]:
X_arc8

In [ ]:
Y_arc8

## Random Forest (COM TEMPO)

In [ ]:
X_arc8

In [ ]:
Y_arc8 = LabelEncoder().fit_transform(Y_arc8)

Y_arc8

In [ ]:
X_arc8_train, X_arc8_test, y_arc8_train, y_arc8_test = train_test_split(X_arc8, Y_arc8, test_size = 0.3, random_state = 0)

RFC_arc8 = RandomForestClassifier(max_depth= 25, n_estimators = 50, random_state = 0)
RFC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = RFC_arc8.predict(X_arc8_test)

In [ ]:
train_score_rf=RFC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
RFC_arc8.feature_importances_

In [ ]:
np.max(RFC_arc8.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc8_train.columns,
    "Imp": RFC_arc8.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,RFC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,RFC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc8 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 8:")

grid_arc8 = GridSearchCV(RFC_arc8 ,params_arc8, scoring='accuracy',cv=5)
grid_arc8.fit(X_arc8_train,y_arc8_train)

print(grid_arc8.best_params_, grid_arc8.best_score_)
rf_best_arc8 = grid_arc8.best_estimator_

y_arc8_pred = rf_best_arc8.predict(X_arc8_test)

accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

print(rf_best_arc8.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc8_train.columns,
    "Imp": rf_best_arc8.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (COM TEMPO)

In [ ]:
X_arc8

In [ ]:
Y_arc8 = LabelEncoder().fit_transform(Y_arc8)

Y_arc8

In [ ]:
X_arc8_train, X_arc8_test, y_arc8_train, y_arc8_test = train_test_split(X_arc8, Y_arc8, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc8 = SVC(kernel = 'linear', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc8 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc8 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc8 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc8_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc8_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc8_SVC_Linear,params_arc8_SVC_RS]:
  print("Otimização Dados Arquitetura 8:")
  grid_arc8_SVC = GridSearchCV(SVC_arc8 ,params, scoring='accuracy')
  grid_arc8_SVC.fit(X_arc8_train,y_arc8_train)
  print(grid_arc8_SVC.best_params_, grid_arc8_SVC.best_score_)
  rf_best_arc8_SVC = grid_arc8_SVC.best_estimator_

  y_arc8_pred_SVC = rf_best_arc8_SVC.predict(X_arc8_test)

  accuracy_score(y_arc8_test, y_arc8_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred_SVC)*100)
  print()

## Tabela sem Tempo

In [ ]:
df_arc8

In [ ]:
df_arc8.drop(c_X_tempo[1:],axis=1,inplace=True)

In [ ]:
df_arc8

In [ ]:
df_arc8.to_csv('DataframeML_ARC8_FFT.csv')

In [ ]:
# Nome das Colunas:
#Lista com Nome de cada Coluna:
coln_arc8=list(df_arc8)
print(coln_arc8, "\n")

print(len(coln_arc8))
#Definindo as Features (X's):
features=coln_arc8[1:]
X_arc8=df_arc8[features]

print(coln_arc8[1:])
#Definindo os Targets (Y's):

targets=coln_arc8[0] #Definindo lista de targets
Y_arc8=df_arc8[targets]

In [ ]:
Y_arc8

In [ ]:
X_arc8

## Random Forest (SEM TEMPO)

In [ ]:
X_arc8

In [ ]:
Y_arc8 = LabelEncoder().fit_transform(Y_arc8)

Y_arc8

In [ ]:
X_arc8_train, X_arc8_test, y_arc8_train, y_arc8_test = train_test_split(X_arc8, Y_arc8, test_size = 0.3, random_state = 0)

RFC_arc8 = RandomForestClassifier(max_depth= 10, n_estimators = 25, random_state = 0)
RFC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = RFC_arc8.predict(X_arc8_test)

In [ ]:
train_score_rf=RFC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_rf)

In [ ]:
test_score_rf=RFC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_rf)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
RFC_arc8.feature_importances_

In [ ]:
np.max(RFC_arc8.feature_importances_)

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_arc8_train.columns,
    "Imp": RFC_arc8.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,RFC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,RFC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV:

In [ ]:
#https://www.kdnuggets.com/2022/10/hyperparameter-tuning-grid-search-random-search-python.html
params_arc8 = {
'n_estimators': [10,25,50,100,200],
'max_depth' : [5,10,25,50,100],
'criterion' :['gini', 'entropy'],
}

In [ ]:
print("Otimização Dados Arquitetura 8:")

grid_arc8 = GridSearchCV(RFC_arc8 ,params_arc8, scoring='accuracy',cv=5)
grid_arc8.fit(X_arc8_train,y_arc8_train)

print(grid_arc8.best_params_, grid_arc8.best_score_)
rf_best_arc8 = grid_arc8.best_estimator_

y_arc8_pred = rf_best_arc8.predict(X_arc8_test)

accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

print(rf_best_arc8.feature_importances_)

imp_df = pd.DataFrame({
    "Varname": X_arc8_train.columns,
    "Imp": rf_best_arc8.feature_importances_
})

In [ ]:
#Vendo a Importância das Features para RF Grid:
imp_df.sort_values(by="Imp", ascending=False)[0:50]

## SVC (SEM TEMPO)

In [ ]:
X_arc8

In [ ]:
Y_arc8 = LabelEncoder().fit_transform(Y_arc8)

Y_arc8

In [ ]:
X_arc8_train, X_arc8_test, y_arc8_train, y_arc8_test = train_test_split(X_arc8, Y_arc8, test_size = 0.3, random_state = 0)

### Linear

In [ ]:
SVC_arc8 = SVC(kernel = 'linear', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Polinomial:

In [ ]:
SVC_arc8 = SVC(kernel = 'poly',degree = 10, random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### RBF:

In [ ]:
SVC_arc8 = SVC(kernel = 'rbf', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### Sigmoid:

In [ ]:
SVC_arc8 = SVC(kernel = 'sigmoid', random_state = 0)
SVC_arc8.fit(X_arc8_train, y_arc8_train)

In [ ]:
y_arc8_pred = SVC_arc8.predict(X_arc8_test)

In [ ]:
train_score_svc=SVC_arc8.score(X_arc8_train,y_arc8_train)
print(train_score_svc)

In [ ]:
test_score_svc=SVC_arc8.score(X_arc8_test,y_arc8_test)
print(test_score_svc)

In [ ]:
accuracy_score(y_arc8_test, y_arc8_pred)
print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred)*100)

In [ ]:
confmatrix_train = confusion_matrix(y_arc8_train,SVC_arc8.predict(X_arc8_train))
confmatrix_test=confusion_matrix(y_arc8_test,SVC_arc8.predict(X_arc8_test))

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_train, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Train set Confusion Matrix',y=1)
plt.show()

fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(confmatrix_test, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(falhas,rotation=90)
ax.yaxis.set_ticklabels(falhas,rotation=0)
ax.set_title('Test set Confusion Matrix',y=1)
plt.show()

### GridSearchCV

Abandonamos o Kernel polinomial, pois não importa o grau do polinomio, sempre fica ruim

In [ ]:
params_arc8_SVC_Linear = {
'kernel':['linear'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

params_arc8_SVC_RS = {
'kernel':['rbf','sigmoid'],
'gamma':['scale', 'auto'],
'C':[0.001,0.01,0.1,1,10,100,1000],
}

In [ ]:
for params in [params_arc8_SVC_Linear,params_arc8_SVC_RS]:
  print("Otimização Dados Arquitetura 8:")
  grid_arc8_SVC = GridSearchCV(SVC_arc8 ,params, scoring='accuracy')
  grid_arc8_SVC.fit(X_arc8_train,y_arc8_train)
  print(grid_arc8_SVC.best_params_, grid_arc8_SVC.best_score_)
  rf_best_arc8_SVC = grid_arc8_SVC.best_estimator_

  y_arc8_pred_SVC = rf_best_arc8_SVC.predict(X_arc8_test)

  accuracy_score(y_arc8_test, y_arc8_pred_SVC)
  print('accuracy_score %:',accuracy_score(y_arc8_test, y_arc8_pred_SVC)*100)
  print()